In [1]:
import numpy as np
import pandas as pd

#done
def convert_to_float(not_float):
    float_list = []
    for i in not_float:
            float_list.append([float(j) for j in i])
    return float_list

In [2]:
#TODO: split  test with training set
def process_file(raw_file_name):
    raw_file = open(raw_file_name, 'r')
    str_data = []
    with open(raw_file_name) as file:
        raw_data = file.readlines()
    str_data = [i.split() for i in raw_data]
    return convert_to_float(str_data)

In [3]:
#done
def take_out_classification(data):
    classfication = []
    for i in data:
        classfication.append(i.pop(0))
    return classfication

In [4]:
#done
def add_back_classification(data, list_of_classfication):
    for i in range(0, len(data)):
        data[i].insert(0, float(list_of_classfication[i]))

In [5]:
#done
def getmean(data, features, instances):
    mean_list = []
    column = 0
    #classif = take_out_classification(data)
    for i in range(features):
        mean = sum(row[column] for row in data)/instances
        column = column + 1
        mean_list.append(mean)
    #add_back_classification(data, classif)
    return mean_list

In [6]:
#done
import math
def getstd(data, mean_list, features, instances):
    standard_d_list = []
    #classif = take_out_classification(data)
    mean_list = getmean(data, features, instances)
    for i in range(features):
        standard_d = math.sqrt((sum(pow((row[i] - mean_list[i]), 2) for row in data)) / instances)
        standard_d_list.append(standard_d)
    #add_back_classification(data, classif)
    #df = pd.DataFrame(standard_d_list)
    #print(df)
    return standard_d_list

In [7]:
#done
def scale(data):
    classif = take_out_classification(data)
    features = len(data[0])
    instances = len(data)
    #print(instances)
    mean_list = getmean(data, features , instances)
    standard_d_list = getstd(data, mean_list, features, instances )
    #df  = pd.DataFrame(data)
    #print(df)
    
    for i in range(0, instances):
        for j in range(features):
            #print("in loop")
            #print(data[i][j])
            data[i][j] = (data[i][j] - mean_list[j])/ standard_d_list[j]
    add_back_classification(data, classif)
    return data

In [8]:
trainSet = [[2.0000000e+00 ,2.8621318e+00,   3.8345958e+00,   3.8510244e+00,   2.5354832e+00,   1.9743413e+00,   2.4662208e+00,   1.0958087e+00,   2.1801037e+00,   2.6330608e+00,   3.1483889e+00], [1.0000000e+00  ,2.0789125e+00, 3.3845572e+00   ,4.0889981e+00   ,3.6589007e+00   ,3.3362607e+00   ,4.9383758e+00   ,1.9881081e+00   ,3.6009237e+00   ,3.1205240e+00  ,1.3965097e+00], [ 2.0000000e+00, 2.2040188e+00,   1.2717340e+00,   3.1890461e+00,   2.9330737e+00,   3.5206010e+00,   2.5425605e+00,   2.2630886e+00,   3.6348978e+00,   3.5018469e+00,   4.1835761e+00], [1.0000000e+00, 2.2040188e+00, 1.2717340e+00, 3.1890461e+00, 2.9330737e+00, 3.5206010e+00, 2.5425605e+00, 2.2630886e+00, 3.6348978e+00, 3.5018469e+00, 4.1835761e+00], [2.0000000e+00  , 3.2174703e+00  , 4.1419433e+00  , 1.6861336e+00 ,  3.0024740e+00  , 4.1632536e+00  , 2.0213114e+00  , 3.7500223e+00  , 2.4615493e+00 ,  2.4107511e+00 ,2.3383039e+00]]

df  = pd.DataFrame(trainSet)
print(df)
trainSet = scale(trainSet)
df  = pd.DataFrame(trainSet)
print(df)
#print(len(trainSet[0]) - 1)
#print(len(trainSet))

    0         1         2         3         4         5         6         7   \
0  2.0  2.862132  3.834596  3.851024  2.535483  1.974341  2.466221  1.095809   
1  1.0  2.078912  3.384557  4.088998  3.658901  3.336261  4.938376  1.988108   
2  2.0  2.204019  1.271734  3.189046  2.933074  3.520601  2.542561  2.263089   
3  1.0  2.204019  1.271734  3.189046  2.933074  3.520601  2.542561  2.263089   
4  2.0  3.217470  4.141943  1.686134  3.002474  4.163254  2.021311  3.750022   

         8         9         10  
0  2.180104  2.633061  3.148389  
1  3.600924  3.120524  1.396510  
2  3.634898  3.501847  4.183576  
3  3.634898  3.501847  4.183576  
4  2.461549  2.410751  2.338304  
    0         1         2         3         4         5         6         7   \
0  2.0  0.780949  0.839207  0.776422 -1.315122 -1.842117 -0.420668 -1.376895   
1  1.0 -0.972539  0.480773  1.060605  1.781452  0.046098  1.964636 -0.332355   
2  2.0 -0.692449 -1.201987 -0.014096 -0.219208  0.301674 -0.347010 -0.01045

In [9]:
data1 = [[ 2.0000000e+00 ,2.8621318e+00,   3.8345958e+00,   3.8510244e+00,   2.5354832e+00,   1.9743413e+00,   2.4662208e+00,   1.0958087e+00,   2.1801037e+00,   2.6330608e+00,   3.1483889e+00],  [1.0000000e+00, 2.2040188e+00,   1.2717340e+00,   3.1890461e+00,   2.9330737e+00,   3.5206010e+00,   2.5425605e+00,   2.2630886e+00,   3.6348978e+00,   3.5018469e+00,   4.1835761e+00]]

print("me ")
#print(data1)
#print(getmean(data1, 2, 2))
#print(getstd(data1,getmean(data1, 2, 2),2, 2))
print(scale(data1))
#print(data1)

me 
[[2.0, 0.9999999999999993, 1.0, 0.9999999999999993, -0.9999999999999989, -0.9999999999999999, -1.0, -1.0, -1.0, -0.9999999999999994, -1.0000000000000004], [1.0, -1.0000000000000007, -1.0, -1.0000000000000007, 1.000000000000001, 1.0000000000000002, 1.0, 1.0, 1.0, 1.0000000000000004, 0.9999999999999996]]


In [10]:
#done
def euclideanDistance(instance_a, instance_b, features):
    #remove classfication for calculation
    #print(instance_a)
    classification_1 = instance_a.pop(0)
    classification_2 = instance_b.pop(0)
    
    distance = 0
    #print(repr(instance_a) +"\n"+ repr(instance_b))
    for i in range(len(features)):
        #print(instance_b)
        #print(features[i])
        #print(instance_b[features[i]])
       # print("instance a = " +  repr(instance_a[i]) + " and instance b = " +  repr(instance_b[i]))
        distance = distance + pow((instance_a[features[i]] - instance_b[features[i]]), 2)
       # print("and the distance is: " + repr(distance) + "\n")
    
    #add classification back
    instance_a.insert(0, classification_1)
    instance_b.insert(0, classification_2)
    return math.sqrt(distance)

In [11]:
data1 = [ 2.0000000e+00 ,2.8621318e+00,   3.8345958e+00,   3.8510244e+00,   2.5354832e+00,   1.9743413e+00,   2.4662208e+00,   1.0958087e+00,   2.1801037e+00,   2.6330608e+00,   3.1483889e+00]
data2 = [2.0000000e+00, 2.0789125e+00, 3.3845572e+00   ,4.0889981e+00   ,3.6589007e+00   ,3.3362607e+00   ,4.9383758e+00   ,1.9881081e+00   ,3.6009237e+00   ,3.1205240e+00  ,1.3965097e+00]
distance = euclideanDistance(data1, data2, [0])
print( 'Distance: ' + repr(distance))

Distance: 0.7832192999999998


In [245]:
def nearest_neighbor(data, test, features):
    best_neighbor = [-1, -1]
    distance = float('inf')
    test_distance = float('inf')
    for i in data:
        if len(features) > 0:
            test_distance = euclideanDistance(i, test, features)
        if test_distance < distance:
            distance = test_distance
            #print(i[0])
            best_neighbor[0] = i[0]
            #print(test_distance)
            best_neighbor[1]= test_distance
    return best_neighbor
            

In [13]:
trainSet = [[2.0000000e+00 ,2.8621318e+00,   3.8345958e+00,   3.8510244e+00,   2.5354832e+00,   1.9743413e+00,   2.4662208e+00,   1.0958087e+00,   2.1801037e+00,   2.6330608e+00,   3.1483889e+00], [1.0000000e+00  ,2.0789125e+00, 3.3845572e+00   ,4.0889981e+00   ,3.6589007e+00   ,3.3362607e+00   ,4.9383758e+00   ,1.9881081e+00   ,3.6009237e+00   ,3.1205240e+00  ,1.3965097e+00]]
testInstance = [1.0000000e+00  ,2.0789125e+00, 3.3845572e+00   ,4.0889981e+00   ,3.6589007e+00   ,3.3362607e+00   ,4.9383758e+00   ,1.9881081e+00   ,3.6009237e+00   ,3.1205240e+00  ,1.3965097e+00]
k = 1
neighbor = nearest_neighbor(trainSet, testInstance, [])
print(neighbor)


[-1, -1]


In [301]:
import numpy as np
def compute_accuracy(data,feature_set, feature):

    #print(feature)
    list_feature = list(feature_set)
    accuracy = 0
    test = []
    neighbor = []
    if feature < 0:
        feature_set.discard(abs(feature))
        list_feature = list(feature_set)
        feature_set.add(abs(feature))
    elif feature > 0:
        feature_set.add(abs(feature))
        list_feature = list(feature_set)
        feature_set.discard(abs(feature))
    #random(data)
    for i in range(660):
        np.random.shuffle(data)
        #print(data)
        test = data.pop(0)
        #print("test " + repr(test[0]))
        neighbor = nearest_neighbor(data, test, list_feature)
        #print("Neighbor " + repr(neighbor))
        #print("found " + repr(nearest_neighbor(data, test, feature)[0]))
        if  neighbor[0] == test[0]:
            accuracy = accuracy + 1
        data.append(test)
    accuracy = accuracy/661
    return accuracy * 100
        

In [227]:
trainSet = [[2.0000000e+00 ,2.8621318e+00,   3.8345958e+00,   3.8510244e+00,   2.5354832e+00,   1.9743413e+00,   2.4662208e+00,   1.0958087e+00,   2.1801037e+00,   2.6330608e+00,   3.1483889e+00], [1.0000000e+00  ,2.0789125e+00, 3.3845572e+00   ,4.0889981e+00   ,3.6589007e+00   ,3.3362607e+00   ,4.9383758e+00   ,1.9881081e+00   ,3.6009237e+00   ,3.1205240e+00  ,1.3965097e+00], [ 2.0000000e+00, 2.2040188e+00,   1.2717340e+00,   3.1890461e+00,   2.9330737e+00,   3.5206010e+00,   2.5425605e+00,   2.2630886e+00,   3.6348978e+00,   3.5018469e+00,   4.1835761e+00], [1.0000000e+00, 2.2040188e+00, 1.2717340e+00, 3.1890461e+00, 2.9330737e+00, 3.5206010e+00, 2.5425605e+00, 2.2630886e+00, 3.6348978e+00, 3.5018469e+00, 4.1835761e+00], [2.0000000e+00  , 3.2174703e+00  , 4.1419433e+00  , 1.6861336e+00 ,  3.0024740e+00  , 4.1632536e+00  , 2.0213114e+00  , 3.7500223e+00  , 2.4615493e+00 ,  2.4107511e+00 ,2.3383039e+00]]
accuracy = 0
accuracy = compute_accuracy(trainSet, [])
print(accuracy)

TypeError: compute_accuracy() missing 1 required positional argument: 'feature'

In [321]:
def forward_selection(data):
    print("Froward selection")
    features = len(data[0]) - 1
    feature_set = set()
    best_accuracy = 0
    #print(best_accuracy)
    for i in range(features):
        print("At level: " + repr(i+1))
        add_this = -1
        for j in range(i, features):
            if j not in feature_set:
                accuracy = compute_accuracy(data, feature_set, j)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    add_this = j
        if add_this > 0:
            feature_set.add(add_this)
            print("Best accuracy so far: " + "%.2f" % best_accuracy + " %" + " With feature set: " + repr(feature_set))
        else:
            print("(Warning, Accuracy has decreased! Continuing search in case of local maxima)")
    print("This is what I got: ")
    print("With feature set: " + repr(feature_set)+ " Best accuracy: " + "%.2f" % best_accuracy + " %")
            
            

In [322]:
data = [[2.0000000e+00 ,2.8621318e+00,   3.8345958e+00,   3.8510244e+00,   2.5354832e+00,   1.9743413e+00,   2.4662208e+00,   1.0958087e+00,   2.1801037e+00,   2.6330608e+00,   3.1483889e+00], [1.0000000e+00  ,2.0789125e+00, 3.3845572e+00   ,4.0889981e+00   ,3.6589007e+00   ,3.3362607e+00   ,4.9383758e+00   ,1.9881081e+00   ,3.6009237e+00   ,3.1205240e+00  ,1.3965097e+00], [ 2.0000000e+00, 2.2040188e+00,   1.2717340e+00,   3.1890461e+00,   2.9330737e+00,   3.5206010e+00,   2.5425605e+00,   2.2630886e+00,   3.6348978e+00,   3.5018469e+00,   4.1835761e+00], [1.0000000e+00, 2.2040188e+00, 1.2717340e+00, 3.1890461e+00, 2.9330737e+00, 3.5206010e+00, 2.5425605e+00, 2.2630886e+00, 3.6348978e+00, 3.5018469e+00, 4.1835761e+00], [2.0000000e+00  , 3.2174703e+00  , 4.1419433e+00  , 1.6861336e+00 ,  3.0024740e+00  , 4.1632536e+00  , 2.0213114e+00  , 3.7500223e+00  , 2.4615493e+00 ,  2.4107511e+00 ,2.3383039e+00]]


trainSet = [[2.0000000e+00, 2.8621318e+00, 3.8345958e+00, 3.8510244e+00, 2.5354832e+00],
            [1.0000000e+00, 2.0789125e+00, 3.3845572e+00, 4.0889981e+00, 3.6589007e+00], 
            [2.0000000e+00, 2.2040188e+00, 1.2717340e+00, 3.1890461e+00, 2.9330737e+00], 
            [1.0000000e+00, 2.2040188e+00, 1.2717340e+00, 3.1890461e+00, 2.9330737e+00],
            [2.0000000e+00, 3.2174703e+00, 4.1419433e+00, 1.6861336e+00, 3.0024740e+00]]

#df  = pd.DataFrame(trainSet)
#print(df)
trainSet = scale(trainSet)
#print(trainSet)

#df  = pd.DataFrame(trainSet)
#print(df)

forward_selection(trainSet)

Froward selection
At level: 1
Best accuracy so far: 40.24 % With feature set: {1}
At level: 2
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 3
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 4
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
This is what I got: 
With feature set: {1} Best accuracy: 40.24 %


In [323]:
def backward_selection(data):
    print("Backward selection")
    features = len(data[0]) - 1
    feature_set = set(i for i in range(features))
    #print(feature_set)
    best_accuracy = 0
    #print(best_accuracy)
    for i in range(features):
        print("At level: " + repr(i+1))
        remove_this = -1
        for j in range(features):
            if j in feature_set:
                accuracy = compute_accuracy(data, feature_set, -1*j)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    remove_this = j
        if remove_this > 0:
            feature_set.discard(remove_this)
            print("Best accuracy so far: " + "%.2f" % best_accuracy + " %" + " With feature set: " + repr(feature_set))
        else:
            print("(Warning, Accuracy has decreased! Continuing search in case of local maxima)")
    print("This is what I got: ")
    print("With feature set: " + repr(feature_set)+ " Best accuracy: " + "%.2f" % best_accuracy + " %")

In [ ]:
def custom_algorithm(data):
    
    

In [324]:
raw_file_open = 'CS170Smalltestdata__37.txt'
data = process_file(raw_file_open)
scaled_data = scale(data)

forward_selection(scaled_data)
backward_selection(scaled_data)

#nearest_neighbor(data, test, )
#print(data)
#print(data[0][1])
#print(data[1])


Froward selection
At level: 1
Best accuracy so far: 83.06 % With feature set: {4}
At level: 2
Best accuracy so far: 88.05 % With feature set: {4, 8}
At level: 3
Best accuracy so far: 91.38 % With feature set: {3, 4, 8}
At level: 4
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 5
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 6
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 7
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 8
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 9
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 10
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
This is what I got: 
With feature set: {3, 4, 8} Best accuracy: 91.38 %
Backward selection
At level: 1
Best accuracy so far: 74.43 % With feature set: {0, 1,